# Make a dataset of vectorized phone front view
## output:
## original SVG (from shape grammar): in iphone_original & samsung_original folder
## rasterized bmp (for resampling): in iphone_pixel & samsung_pixel folder
## resampled SVG (actually used for training): in iphone_resample % samsung_resample folder
----------------could skip the above process if download [SVG-data.zip](https://drive.google.com/file/d/1EHMhK5YhudFL1mEfMslS2hXQ92VLCOcK/view?usp=sharing)----------------
## GNN-readable SVG (saved in pickle format): in pkl/data, label, curve_label, dist_mat for train/test/reference set
----------------could skip the above process if download [GNN-data.zip](https://drive.google.com/file/d/1FgSsBIPzOgKaXGAYXavOU56oW8111gu4/view?usp=sharing)----------------


In [2]:
import util as u

#### 1. Generate SVG using shape grammar & resample
-------could skip this step if download [SVG-data.zip](https://drive.google.com/file/d/1EHMhK5YhudFL1mEfMslS2hXQ92VLCOcK/view?usp=sharing)--------

In [5]:
folder_list=[]
original_iphone_folder="iphone_original"
folder_list.append(original_iphone_folder)
original_samsung_folder="samsung_original"
folder_list.append(original_samsung_folder)
pixel_iphone_folder = "iphone_pixel"
folder_list.append(pixel_iphone_folder)
pixel_samsung_folder = "samsung_pixel"
folder_list.append(pixel_samsung_folder)
resample_iphone_folder = "iphone_resample"
folder_list.append(resample_iphone_folder)
resample_samsung_folder = "samsung_resample"
folder_list.append(resample_samsung_folder)
pkl_folder="pkl"
folder_list.append(pkl_folder)
        
u.initialize(folder_list)

In [20]:
# pkl_folder="pkl"
iphone_params_path="%s/iphone_params.pkl"%pkl_folder
samsung_params_path="%s/samsung_params.pkl"%pkl_folder

data_size=(1,10000)
iphone_params=u.create_iphone_params(data_size)
u.dump_item(iphone_params, iphone_params_path)
samsung_params=u.create_samsung_params(data_size)
u.dump_item(samsung_params, samsung_params_path)

In [21]:
u.create_iphone_dataset(iphone_params, \
                    original_iphone_folder, \
                    stroke_aug=0, \
                    stripe=0)

u.create_samsung_dataset(samsung_params, \
                    original_samsung_folder, \
                    stroke_aug=0, \
                    stripe=0)

In [1]:
pixel_format=".bmp"
u.resample(original_iphone_folder, pixel_iphone_folder,\
          pixel_format, resample_iphone_folder)
u.resample(original_samsung_folder, pixel_samsung_folder,\
          pixel_format, resample_samsung_folder)

#### 2. train/test split and process to be GNN-friendly
-------could skip this step if download [GNN-data.zip](https://drive.google.com/file/d/1FgSsBIPzOgKaXGAYXavOU56oW8111gu4/view?usp=sharing)-------

In [6]:
train_ratio=0.9
train_iphones, test_iphones=u.split(resample_iphone_folder, train_ratio)
train_samsungs, test_samsungs=u.split(resample_samsung_folder, train_ratio)

In [9]:
#no augment
train_list, train_label, train_cor_list, train_name_list=\
u.get_normalized_data(train_iphones, train_samsungs, h_aug=0, v_aug=0)

test_list, test_label, test_cor_list, test_name_list=\
u.get_normalized_data(test_iphones, test_samsungs, h_aug=0, v_aug=0)

train_set_name = "%s/train_data_no_aug.pkl"%pkl_folder
train_curve_label_name="%s/train_curve_label_no_aug.pkl"%pkl_folder
train_label_name= "%s/train_label_no_aug.pkl"%pkl_folder
train_dist_mat_name="%s/train_dist_mat_no_aug.pkl"%pkl_folder

test_set_name = "%s/test_data_no_aug.pkl"%pkl_folder
test_curve_label_name="%s/test_curve_label_no_aug.pkl"%pkl_folder
test_label_name= "%s/test_label_no_aug.pkl"%pkl_folder
test_dist_mat_name="%s/test_dist_mat_no_aug.pkl"%pkl_folder

u.save_dataset(train_set_name, train_curve_label_name, train_label_name, train_dist_mat_name,
             test_set_name, test_curve_label_name, test_label_name, test_dist_mat_name,
             train_list, train_label, train_cor_list,
             test_list, test_label, test_cor_list)

In [12]:
'''make source validation set(no stripe)'''
ref_folder_list=[]

ref_original_iphone_folder="ref_iphone_original"#already here!
ref_original_samsung_folder="ref_samsung_original"#already here!

ref_pixel_iphone_folder = "ref_iphone_pixel"
ref_folder_list.append(ref_pixel_iphone_folder)
ref_pixel_samsung_folder = "ref_samsung_pixel"
ref_folder_list.append(ref_pixel_samsung_folder)

ref_resample_iphone_folder="ref_iphone_resample"
ref_folder_list.append(ref_resample_iphone_folder)
ref_resample_samsung_folder="ref_samsung_resample"
ref_folder_list.append(ref_resample_samsung_folder)

u.initialize(ref_folder_list)

pixel_format=".bmp"
u.resample(ref_original_iphone_folder, ref_pixel_iphone_folder,\
          pixel_format, ref_resample_iphone_folder)
u.resample(ref_original_samsung_folder, ref_pixel_samsung_folder,\
          pixel_format, ref_resample_samsung_folder)

ref_iphones = u.get_filelist(dir, ref_resample_iphone_folder)
ref_samsungs = u.get_filelist(dir, ref_resample_samsung_folder)

ref_list, ref_label, ref_cor_list, ref_name_list=\
u.get_normalized_data(ref_iphones, ref_samsungs, h_aug=0, v_aug=0)

ref_set_name = "%s/ref_data.pkl"%pkl_folder
ref_curve_label_name="%s/ref_curve_label.pkl"%pkl_folder
ref_label_name= "%s/ref_label.pkl"%pkl_folder
ref_dist_mat_name="%s/ref_dist_mat.pkl"%pkl_folder

u.save_set(ref_set_name, ref_curve_label_name, ref_label_name, \
         ref_dist_mat_name, ref_list, ref_label, ref_cor_list)